## Prediction with everything

In [1]:
import pandas as pd
import re

%matplotlib inline

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
csvs = pd.read_csv('../data/master.csv', index_col='SK_ID_CURR')
sql = pd.read_csv('../data/processed/full_sql.csv', index_col='SK_ID_CURR')
api = pd.read_csv('../data/api_data_clean.csv', index_col='sk_id_curr')

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22,23,24,32,33,34,44,46,59,64,116) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
csvs.head(1)

In [ ]:
sql.head(1)

In [ ]:
api.head(1)

In [ ]:
temp_1 = pd.merge(sql, csvs, how='left', right_index=True, left_index=True)

In [ ]:
master = pd.merge(temp_1, api, how='left', right_index=True, left_index=True)

In [ ]:
master.head()

## Create the training and test datasets

In [ ]:
train = master[~master.TARGET.isnull()]

In [ ]:
test = master.reindex(master.index.difference(train.index))

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
a = []
for col in master.columns:
    if master[col].isnull().sum() > 50000:
        a.append(col)

print(a)

## Flags

In [ ]:
flags = []
pattern = re.compile("FLAG|flag")
for col in master.columns:
    if pattern.search(col):
        flags.append(col)

In [ ]:
drop_flags= ['FLAG_DOCUMENT_11_x', 'FLAG_DOCUMENT_14.1', 'FLAG_DOCUMENT_16_y',
'FLAG_DOCUMENT_18_y', 'FLAG_DOCUMENT_3_y', 'FLAG_DOCUMENT_4_y', 'FLAG_DOCUMENT_6_y', 
            'FLAG_OWN_CAR_y', 'FLAG_OWN_REALTY_y']


In [ ]:
FLAG_DOCUMENT_7 = pd.to_numeric(master['FLAG_DOCUMENT_7_y'] + master['FLAG_DOCUMENT_7_x'])

In [ ]:
FLAG_DOCUMENT_8 = pd.to_numeric(master['FLAG_DOCUMENT_8_y'] + master['FLAG_DOCUMENT_8_x'])

In [ ]:
def prepare_flags(_df):
    _df['FLAG_DOCUMENT_7'] = pd.to_numeric(_df['FLAG_DOCUMENT_7_y'] + _df['FLAG_DOCUMENT_7_x'])
    _df['FLAG_DOCUMENT_8'] = pd.to_numeric(_df['FLAG_DOCUMENT_8_y'] + _df['FLAG_DOCUMENT_8_x'])

    # These are duplicate 
    dupes_flags= ['FLAG_DOCUMENT_11_x', 'FLAG_DOCUMENT_14.1', 'FLAG_DOCUMENT_16_y',
                'FLAG_DOCUMENT_18_y', 'FLAG_DOCUMENT_3_y', 'FLAG_DOCUMENT_4_y', 'FLAG_DOCUMENT_6_y', 
                'FLAG_OWN_CAR_y', 'FLAG_OWN_REALTY_y','FLAG_DOCUMENT_7_y','FLAG_DOCUMENT_7_x',
                'FLAG_DOCUMENT_8_y','FLAG_DOCUMENT_8_x']    
    
    _df = _df.drop(columns=dupes_flags)
    
    # Columns with a lot of Nans
    drop_cols=['FLAG_OWN_CAR_x','FLAG_DOCUMENT_16_x','FLAG_DOCUMENT_11_y'  ,'FLAG_DOCUMENT_2']
    
    ## Todo instead of dropping Nan maybe interpolate them?
    _df = _df.drop(columns=drop_cols)
    
    # Fixing dtypes
    _df['FLAG_OWN_REALTY_x'] = _df['FLAG_OWN_REALTY_x'].apply(lambda y: 1 if y =='Y' else 0)
    _df['flag_document_9'] = _df['flag_document_9'].astype(int)
    _df['flag_document_13'] = _df['flag_document_13'].astype(int)
    
    return _df

In [ ]:
master_flags = prepare_flags(master)

In [ ]:
flags = []
pattern = re.compile("FLAG|flag")
for col in master_flags.columns:
    if pattern.search(col):
        flags.append(col)

## X and Y

In [ ]:
not_flags = list(set(master_flags.columns) - set(flags))
not_flags.sort()

In [ ]:
def prepare_x_y_merge(_df):
    combine = ['AMT_CREDIT_x', 'AMT_INCOME_TOTAL_x', 'DEF_30_CNT_SOCIAL_CIRCLE_x', 
              'FLOORSMAX_AVG_x', 'LIVINGAPARTMENTS_AVG_x', 'OBS_60_CNT_SOCIAL_CIRCLE_x',
              'YEARS_BEGINEXPLUATATION_AVG_x' ]
    
    _df['AMT_CREDIT'] = pd.to_numeric(_df['AMT_CREDIT_x'] + _df['AMT_CREDIT_y'])
    _df['AMT_INCOME_TOTAL'] = pd.to_numeric(_df['AMT_INCOME_TOTAL_x'] + _df['AMT_INCOME_TOTAL_y'])
    _df['DEF_30_CNT_SOCIAL_CIRCLE'] = pd.to_numeric(_df['DEF_30_CNT_SOCIAL_CIRCLE_x'] + _df['DEF_30_CNT_SOCIAL_CIRCLE_y'])
    _df['FLOORSMAX_AVG'] = pd.to_numeric(_df['FLOORSMAX_AVG_x'] + _df['FLOORSMAX_AVG_y'])
    _df['LIVINGAPARTMENTS_AVG'] = pd.to_numeric(_df['LIVINGAPARTMENTS_AVG_x'] + _df['LIVINGAPARTMENTS_AVG_y'])
    _df['OBS_60_CNT_SOCIAL_CIRCLE'] = pd.to_numeric(_df['OBS_60_CNT_SOCIAL_CIRCLE_x'] + _df['OBS_60_CNT_SOCIAL_CIRCLE_y'])
    _df['YEARS_BEGINEXPLUATATION_AVG'] = pd.to_numeric(_df['YEARS_BEGINEXPLUATATION_AVG_x'] + _df['YEARS_BEGINEXPLUATATION_AVG_y'])

    _df = _df.drop(columns=combine)
    # Remove exact duplicates:
    duplicates = ['AMT_REQ_CREDIT_BUREAU_HOUR_y', 
    'CNT_FAM_MEMBERS_x', 'DAYS_EMPLOYED_y', 'NONLIVINGAPARTMENTS_AVG_y',
    'REGION_POPULATION_RELATIVE_y', 'REG_REGION_NOT_WORK_REGION_y',
    'WALLSMATERIAL_MODE_y', 'WEEKDAY_APPR_PROCESS_START_y']
    
    _df = _df.drop(columns=duplicates)
    
    # Low count
    low_count = ['AMT_REQ_CREDIT_BUREAU_DAY_y', 'APARTMENTS_MODE_y', 
                 'FLOORSMAX_MEDI_y', 'NAME_TYPE_SUITE_y', 'NONLIVINGAREA_MEDI_y']
    
    _df = _df.drop(columns=low_count)
    
    # Columns with a lot of Nans
    drop_cols=['AMT_REQ_CREDIT_BUREAU_DAY_x', 'APARTMENTS_MODE_x', 'FLOORSMAX_AVG', 'FLOORSMAX_MEDI_x',
              'LIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_MEDI_x']
    
    ## Todo instead of dropping Nan maybe interpolate them?
    _df = _df.drop(columns=drop_cols)
    
    return _df

In [ ]:
master_dupes = prepare_x_y_merge(master_flags)

In [ ]:
master_dupes.shape

In [ ]:
master_dupes[['OCCUPATION_TYPE', 'OCCUPATION_TYPE.1']].head()

## Dot ones

In [ ]:
dots = []
pattern = re.compile("\.1")
for col in master_dupes.columns:
    if pattern.search(col):
        dots.append(col)

In [ ]:
dots

In [ ]:
duplicate = REGION_RATING_CLIENT_W_CITY.1, EXT_SOURCE_1.1, DAYS_REGISTRATION.1, EXT_SOURCE_3.1,
            LIVINGAREA_MEDI.1, HOUR_APPR_PROCESS_START, FLOORSMIN_MEDI.1, OCCUPATION_TYPE.1
    

In [ ]:
def drop_dot_1(_df):
    dots = ['REGION_RATING_CLIENT_W_CITY.1', 'EXT_SOURCE_1.1', 'DAYS_REGISTRATION.1', 'EXT_SOURCE_3.1',
            'LIVINGAREA_MEDI.1', 'HOUR_APPR_PROCESS_START.1', 'FLOORSMIN_MEDI.1', 'OCCUPATION_TYPE.1']
    
    _df = _df.drop(columns=dots)
    
    return _df

In [ ]:
md = drop_dot_1(master_dupes)

## Nulls

In [ ]:
lots_of_nas = []
for col in md.columns:
    if md[col].isnull().sum() > 25000:
        lots_of_nas.append(col)

print(lots_of_nas)

In [ ]:
md[lots_of_nas].describe()

In [ ]:
def drop_nans(_df):
    lots_of_nans = ['IS_KAEL', 'BASEMENTAREA_AVG', 'ENTRANCES_AVG', 'FONDKAPREMONT_MODE', 
                    'OWN_CAR_AGE', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 
                    'LIVINGAPARTMENTS_MEDI', 'WEEKDAY_APPR_PROCESS_START_x', 
                    'WALLSMATERIAL_MODE_x', 'AMT_REQ_CREDIT_BUREAU_HOUR_x', 
                    'REGION_POPULATION_RELATIVE_x', 'NONLIVINGAPARTMENTS_AVG_x', 
                    'DAYS_EMPLOYED_x', 'CNT_FAM_MEMBERS_y', 'LIVE_REGION_NOT_WORK_REGION', 
                    'REG_REGION_NOT_LIVE_REGION', 'NAME_HOUSING_TYPE', 'NAME_CONTRACT_TYPE', 
                    'LIVINGAPARTMENTS_AVG_y', 'EXT_SOURCE_1', 'LIVINGAREA_MEDI', 'FLOORSMIN_MEDI',
                    'OCCUPATION_TYPE', 'LANDAREA_MEDI', 'LANDAREA_MEDI.1', 'YEARS_BEGINEXPLUATATION_AVG',
                    # Empty?
                    'DEF_30_CNT_SOCIAL_CIRCLE_y', 'OBS_60_CNT_SOCIAL_CIRCLE_y', 'REG_REGION_NOT_WORK_REGION_x',
                   'AMT_CREDIT_y', 'AMT_INCOME_TOTAL_y']
    
    _df = _df.drop(columns=lots_of_nans)
    
    # These have not so many nans but a lot for the target variable
    not_so_many= ['EXT_SOURCE_3','YEARS_BEGINEXPLUATATION_AVG_y','FLOORSMAX_AVG_y']
    _df = _df.drop(columns=not_so_many)
    
    # We are using the average for the others
    means = ['DEF_60_CNT_SOCIAL_CIRCLE',
     'AMT_ANNUITY', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']
    
    for col in means:
        _df[col] = _df[col].fillna(_df[col].mean())
        
    return _df

In [ ]:
mf = drop_nans(md)

## Dtypes

In [ ]:
cats = ['NAME_TYPE_SUITE_x', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'name_education_type', 'name_income_type']

In [ ]:
to_numeric = ['AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT']

In [ ]:
wide_cats = ['ORGANIZATION_TYPE', 'ORGANIZATION_TYPE_CAT']

In [ ]:
def prepare_dtypes(_df):
    interrogation  = ['FLOORSMIN_AVG', 'TOTALAREA_MODE', 'FLOORSMIN_MODE', 'NONLIVINGAREA_AVG', 'HOUSETYPE_MODE', 'LANDAREA_AVG',
    'ELEVATORS_AVG', 'COMMONAREA_MODE']
    
    _df = _df.drop(columns=interrogation)
    
    # categoricals to dummies
    cats = ['NAME_TYPE_SUITE_x', 'CODE_GENDER', 'NAME_FAMILY_STATUS', 'name_education_type', 'name_income_type']
    
    _df = pd.get_dummies(_df, columns=cats, drop_first=True)
    
    #convert
    to_numeric = ['AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'REGION_RATING_CLIENT']
    
    _df['AMT_GOODS_PRICE'] = pd.to_numeric(_df['AMT_GOODS_PRICE'].apply(lambda x: x.split('€')[0]))
    _df['EXT_SOURCE_2'] = pd.to_numeric(_df['EXT_SOURCE_2'].apply(lambda x: x.split('%')[0]), errors='coerce')
    _df.loc[100004,'REGION_RATING_CLIENT'] = 2
    
    #drop wide cats
    wide_cats = ['ORGANIZATION_TYPE', 'ORGANIZATION_TYPE_CAT']
    _df = _df.drop(columns=wide_cats)
    
    return _df

In [ ]:
m_dtypes = prepare_dtypes(mf)

In [ ]:
m_dtypes.dtypes

In [ ]:
m_dtypes.to_csv('../data/processed/master_2.csv')